# Running an Individual Test Plan

This notebook shows how to run an individual test plan and pass custom config parameters for the tests.

In [1]:
%load_ext dotenv
%dotenv dev.env

import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

cannot find .env file


In [2]:
import validmind as vm


vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  project = "clhsvn7va0000kgrl0nwybdff"
)
  
  

Connected to ValidMind. Project: Customer Churn Model dev - Initial Validation (clhsvn7va0000kgrl0nwybdff)


## Load the Demo Dataset

In [3]:
# You can also import customer_churn like this:
from validmind.datasets.classification import customer_churn as demo_dataset
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

### Prepocess the Raw Dataset
We will need to preprocess the dataset and produce the training, test and validation splits first.

In [4]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

## Train a Model for Testing

We train a simple customer churn model for our test.

In [5]:
x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

## Import and Run the Individual Test Plan

### Initialize ValidMind objects

We initize the objects required to run test plans using the ValidMind framework 

In [6]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...
Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


### List of available Test Plans
The interface to show list of test plans available in the ValidMind development framework

In [7]:
vm.test_plans.list_plans()

ID,Name,Description
binary_classifier_metrics,BinaryClassifierMetrics,Test plan for sklearn classifier metrics
binary_classifier_validation,BinaryClassifierPerformance,Test plan for sklearn classifier models
binary_classifier_model_diagnosis,BinaryClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.
time_series_multivariate,TimeSeriesMultivariate,Test plan to perform time series multivariate analysis.
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.
regression_model_description,RegressionModelDescription,Test plan for performance metric of regression model of statsmodels library



### Detail of an individual Test Plan
The interface will get detail of a specific test plan

In [8]:
vm.test_plans.describe_plan("binary_classifier_model_diagnosis")

ID,Name,Description,Required Context,Tests
binary_classifier_model_diagnosis,BinaryClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests,model,"OverfitDiagnosis (ThresholdTest), WeakspotsDiagnosis (ThresholdTest), RobustnessDiagnosis (ThresholdTest)"


### Define the required config parameters
The config can be apply to specific test to override the default configuration parameters.

The format of a config is:
```
config = {
    "<test1_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
     "<test2_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
}
```

Users can input the configuration to test plan using **`config`**, allowing fine-tuning the suite according to their specific data requirements. 

In [9]:
config={
    "overfit_regions": {
        "cut_off_percentage": 3,
        "feature_columns": ["Age", "Balance", "Tenure", "NumOfProducts"]
    },
    "weak_spots":{
        "features_columns": ["Age", "Balance"],
        "accuracy_gap_threshold": 85,
    },
    "robustness":{
        "features_columns": [ "Balance", "Tenure"],
        "scaling_factor_std_dev_list": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        "accuracy_decay_threshold": 4,
    }
}

### Run the test plan and display results

In [10]:
model_diagnosis_test_plan = vm.run_test_plan("binary_classifier_model_diagnosis", 
                                             model=vm_model,
                                             config=config)

### Accessing the test plan results

We can now access all the results of the test plan, including subtest plans using `test_plan.get_results()`.

- `test_plan.get_results()`: With no arguments, this returns a list of all results
- `test_plan.get_results(test_id)`: If provided with a test id, this returns the all results that match the given test id

By default, get_results() returns a list, in case there are multiple tests with the same id.

In [11]:
model_diagnosis_test_plan.get_results()

[TestPlanTestResult(result_id="overfit_regions", test_results),
 TestPlanTestResult(result_id="weak_spots", test_results),
 TestPlanTestResult(result_id="robustness", test_results)]

In [12]:
model_robustness = model_diagnosis_test_plan.get_results("robustness")[0]
model_robustness.show()